In [1]:
import boto3
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm

# Download data from S3
Download most recent death data from s3 into `deathdata.csv`. Skip if you've done this recently it's a big file :) 

In [2]:
session = boto3.Session(
    aws_access_key_id=os.environ["S3_KEY"],
    aws_secret_access_key=os.environ["S3_PRIVATE_KEY"]
)
s3 = session.client('s3')
from pprint import pprint
def hook(t):
  def inner(bytes_amount):
    t.update(bytes_amount)
  return inner

BUCKET_NAME = 'nvss-deaths'
file = sorted([el['Key'] for el in s3.list_objects(Bucket=BUCKET_NAME)['Contents']],reverse=True)[0]

path = "deathdata.csv"
file_object = s3.get_object(Bucket=BUCKET_NAME, Key=file)
filesize = file_object['ContentLength']

with tqdm(total=filesize, unit='B') as t:
    with open(path, 'wb') as f:
        s3.download_fileobj(BUCKET_NAME, file, f, Callback=hook(t))

  0%|          | 0/765827567 [00:00<?, ?B/s]

# Preprocessing
Read in data from `deathdata.csv` and do necessary preprocessing

In [10]:
# Read in the file
dtype = {
    'State' : 'string',
    'Indicator': 'string',
    'COVID-19 Deaths': 'float64',
    'Pneumonia Deaths': 'float64',
    'Pneumonia and COVID-19 Deaths': 'float64',
    'Influenza Deaths': 'float64',
    'Pneumonia, Influenza, or COVID-19 Deaths': 'float64',
    'Total Deaths': 'float64',
    'Percent of Expected Deaths': 'float64',
    'Start week': 'string',
    'End Week': 'string',
    'Start Date': 'string',
    'End Date': 'string',
    'MMWR Week': 'float64',
    'Week Ending Date': 'string',
    'Data as of': 'string'
}
deaths = pd.read_csv('deathdata.csv',dtype=dtype,low_memory=False).drop('Unnamed: 0',axis=1)
deaths

,Unnamed: 0.1,Data as of,Start week,End Week,Group,State,Indicator,COVID-19 Deaths,Total Deaths,Percent of Expected Deaths,...,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote,Start Date,End Date,Year,Month,MMWR Week,Week Ending Date
0,0,05/22/2020,02/01/2020,02/01/2020,by week,United States,Week-ending,0.0,57584.0,0.97,...,0.0,475.0,4188.0,NaN,<NA>,<NA>,NaN,NaN,NaN,<NA>
1,1,05/22/2020,02/08/2020,02/08/2020,by week,United States,Week-ending,1.0,58245.0,0.97,...,0.0,507.0,4223.0,NaN,<NA>,<NA>,NaN,NaN,NaN,<NA>
2,2,05/22/2020,02/15/2020,02/15/2020,by week,United States,Week-ending,0.0,57585.0,0.98,...,0.0,541.0,4288.0,NaN,<NA>,<NA>,NaN,NaN,NaN,<NA>
3,3,05/22/2020,02/22/2020,02/22/2020,by week,United States,Week-ending,2.0,57640.0,0.99,...,0.0,553.0,4165.0,NaN,<NA>,<NA>,NaN,NaN,NaN,<NA>
4,4,05/22/2020,02/29/2020,02/29/2020,by week,United States,Week-ending,5.0,57956.0,1.01,...,3.0,629.0,4358.0,NaN,<NA>,<NA>,NaN,NaN,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4135944,4135944,06/02/2023,<NA>,<NA>,by week,Puerto Rico,<NA>,12.0,588.0,108.00,...,NaN,NaN,90.0,One or more data cells have counts between 1-9...,04/23/2023,04/29/2023,2023,NaN,17.0,04/29/2023
4135945,4135945,06/02/2023,<NA>,<NA>,by week,Puerto Rico,<NA>,13.0,605.0,112.00,...,10.0,NaN,91.0,One or more data cells have counts between 1-9...,04/30/2023,05/06/2023,2023,NaN,18.0,05/06/2023
4135946,4135946,06/02/2023,<NA>,<NA>,by week,Puerto Rico,<NA>,13.0,563.0,103.00,...,NaN,0.0,79.0,One or more data cells have counts between 1-9...,05/07/2023,05/13/2023,2023,NaN,19.0,05/13/2023
4135947,4135947,06/02/2023,<NA>,<NA>,by week,Puerto Rico,<NA>,11.0,441.0,78.00,...,NaN,NaN,69.0,One or more data cells have counts between 1-9...,05/14/2023,05/20/2023,2023,NaN,20.0,05/20/2023


In [11]:
# Change date cols to datetime
date_cols = ['Data as of','End Week','Week Ending Date']
for date_col in date_cols:
    deaths[date_col] = pd.to_datetime(deaths[date_col],errors='coerce')
    
# Merge week ending date and end week (changed field name)
deaths['Weekdate'] = deaths['Week Ending Date'].fillna(deaths['End Week'])

# Sort and drop all the irrelevant columns
deaths = deaths.sort_values(by=['State','Weekdate','Data as of'],ascending=[True,True,True])
deaths = deaths[['State','Weekdate','Data as of','COVID-19 Deaths','Total Deaths']]
deaths['Percent COVID Deaths'] = deaths['COVID-19 Deaths']/deaths['Total Deaths']

# Add map categories from https://covid.cdc.gov/covid-data-tracker/#cases_percent-covid-deaths
deaths['Map Category'] = np.select([
    deaths['Percent COVID Deaths'].isna(), # 0
    deaths['Percent COVID Deaths']< 0.02, # 1
    deaths['Percent COVID Deaths'].between(0.02,.04,inclusive='left'), # 2
    deaths['Percent COVID Deaths'].between(0.04,0.06,inclusive='left'), # 3
    deaths['Percent COVID Deaths'].between(0.06,0.08,inclusive='left'), # 4
    deaths['Percent COVID Deaths'] >= 0.08], # 5
    [0,1,2,3,4,5]
)
deaths

,State,Weekdate,Data as of,COVID-19 Deaths,Total Deaths,Percent COVID Deaths,Map Category
274157,Alabama,2020-01-04,2021-01-06,0.0,630.0,0.0,1
277019,Alabama,2020-01-04,2021-01-07,0.0,1081.0,0.0,1
279881,Alabama,2020-01-04,2021-01-08,0.0,1081.0,0.0,1
282745,Alabama,2020-01-04,2021-01-11,0.0,1081.0,0.0,1
285661,Alabama,2020-01-04,2021-01-12,0.0,1081.0,0.0,1
...,...,...,...,...,...,...,...
4097322,Wyoming,2023-05-27,2023-05-31,0.0,NaN,NaN,0
4106934,Wyoming,2023-05-27,2023-06-01,0.0,14.0,0.0,1
4116546,Wyoming,2023-05-27,2023-06-02,0.0,21.0,0.0,1
4126158,Wyoming,2023-05-27,2023-06-02,0.0,21.0,0.0,1


# Analysis
Look at overall and jurisdiction-level volatility in percent COVID-19 associated deaths

In [34]:
deaths_no_zero = 
deaths_agg = deaths.groupby(['State','Weekdate']).agg({'Map Category' : ['mean','min','max','count'],
                                                       'Percent COVID Deaths' : ['mean','min','max']})

deaths_agg.columns=deaths_agg.columns.to_flat_index()
deaths_agg = deaths_agg.reset_index()
deaths_agg.columns = [str(c).translate({ord(c): None for c in '(),\''}) for c in deaths_agg.columns]
deaths_agg['cat_diff'] = deaths_agg['Map Category max']-deaths_agg['Map Category min']
deaths_agg['percent_diff'] = deaths_agg['Percent COVID Deaths max']-deaths_agg['Percent COVID Deaths min']
deaths_agg.groupby('State').agg({'cat_diff': ['mean','min','max'],'percent_diff': ['mean','min','max']})

cat_diff         percent_diff               
                          mean min max         mean  min       max
State                                                             
Alabama               1.980769   0   5     0.029122  0.0  0.183344
Alaska                1.317308   0   5     0.020305  0.0  0.261999
Arizona               1.024038   0   5     0.030535  0.0  0.273027
Arkansas              1.846154   0   5     0.027935  0.0  0.215660
California            1.081731   0   5     0.034710  0.0  0.327090
Colorado              2.000000   0   5     0.031522  0.0  0.287403
Connecticut           2.591346   0   5     0.074408  0.0  0.733513
Delaware              2.149038   0   5     0.027531  0.0  0.173261
District of Columbia  1.677885   0   5     0.030829  0.0  0.311111
Florida               0.913462   0   5     0.031099  0.0  0.249948
Georgia               2.125000   0   5     0.032763  0.0  0.235685
Hawaii                1.528846   0   5     0.016871  0.0  0.171429
Idaho                 2.326923   0   5     0.028997  0.0  0.258242
Illinois              0.956731   0   5     0.027579  0.0  0.265114
Indiana               1.995192   0   5     0.029850  0.0  0.271031
Iowa                  1.764423   0   5     0.030823  0.0  0.272623
Kansas                1.990385   0   5     0.032932  0.0  0.253283
Kentucky              2.884615   0   5     0.038828  0.0  0.270067
Louisiana             2.437500   0   5     0.030262  0.0  0.166743
Maine                 1.437500   0   5     0.014099  0.0  0.135478
Maryland              1.475962   0   5     0.027044  0.0  0.238112
Massachusetts         1.197115   0   5     0.020833  0.0  0.212005
Michigan              1.072115   0   5     0.022121  0.0  0.199558
Minnesota             1.721154   0   5     0.029011  0.0  0.292434
Mississippi           1.971154   0   5     0.033297  0.0  0.226671
Missouri              1.875000   0   5     0.030802  0.0  0.203227
Montana               2.052885   0   5     0.037206  0.0  0.307947
Nebraska              2.485577   0   5     0.033678  0.0  0.407871
Nevada                2.355769   0   5     0.035668  0.0  0.235161
New Hampshire         1.115385   0   5     0.020030  0.0  0.232143
New Jersey            0.918269   0   5     0.023243  0.0  0.214886
New Mexico            2.663462   0   5     0.034444  0.0  0.266029
New York              1.057692   0   5     0.024142  0.0  0.185348
New York City         0.576923   0   5     0.014961  0.0  0.117438
North Carolina        2.677885   0   5     0.130646  0.0  0.947884
North Dakota          1.798077   0   5     0.035163  0.0  0.265494
Ohio                  1.581731   0   5     0.033498  0.0  0.267869
Oklahoma              2.442308   0   5     0.041321  0.0  0.321799
Oregon                2.379808   0   5     0.023114  0.0  0.141069
Pennsylvania          0.769231   0   5     0.035366  0.0  0.211950
Puerto Rico           2.221154   0   5     0.151973  0.0  0.957929
Rhode Island          1.975962   0   5     0.041318  0.0  0.500000
South Carolina        2.514423   0   5     0.026603  0.0  0.244459
South Dakota          2.052885   0   5     0.047231  0.0  0.334221
Tennessee             1.250000   0   5     0.027812  0.0  0.218824
Texas                 1.375000   0   5     0.037235  0.0  0.280592
United States         0.514423   0   4     0.024991  0.0  0.203142
Utah                  2.120192   0   5     0.026867  0.0  0.235075
Vermont               0.913462   0   5     0.006685  0.0  0.126667
Virginia              1.701923   0   5     0.019980  0.0  0.156766
Washington            2.067308   0   5     0.018616  0.0  0.154589
West Virginia         2.754808   0   5     0.315525  0.0  0.968085
Wisconsin             1.567308   0   5     0.028433  0.0  0.256659
Wyoming               1.533654   0   5     0.037106  0.0  0.380165